In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import requests
import os
from pyspark.sql import SparkSession

class TripDataDownloader:
    """
    Classe responsável por baixar arquivos de dados de viagens de táxi
    de um endpoint HTTP e salvá-los em um caminho de destino local ou em nuvem.

    Parâmetros:
    - spark: SparkSession ativa para leitura e escrita de arquivos Parquet.
    - s3_base_path: Caminho base onde os arquivos serão gravados (pode ser local ou S3).
    - base_url: URL base para download dos arquivos.
    - types: Lista de tipos de táxis (ex: ['yellow', 'green']).
    - months: Lista de meses (formato 'MM') para filtrar os arquivos.
    - years: Lista de anos (formato 'YYYY') opcional para filtrar os arquivos.
    """
    def __init__(self, spark, s3_base_path, base_url, types=None, months=None, years=None):
        """
        Inicializa o downloader com parâmetros de configuração.

        - spark: instância de SparkSession para operações de I/O.
        - s3_base_path: diretório ou bucket onde os dados serão salvos.
        - base_url: prefixo de URL para download dos arquivos de viagem.
        - types: tipos de trips a serem baixados (default: None).
        - months: meses a serem processados (default: None).
        - years: anos a serem processados (default: None).
        """
        self.spark = spark
        self.s3_base_path = s3_base_path.rstrip('/')
        self.base_url = base_url
        self.types = types
        self.months = months
        self.years = years

    def download_and_save(self):
        """
        Para cada combinação de tipo, mês e ano configurados:
        1. Constrói o nome do arquivo e a URL de download.
        2. Faz requisição HTTP para obter o arquivo Parquet.
        3. Armazena temporariamente em '/tmp'.
        4. Lê o Parquet com Spark e escreve no destino definido.
        5. Remove o arquivo temporário ao final.
        """
        spark = SparkSession.builder.appName("iFoodCase").getOrCreate()
        for t in self.types:
            for m in self.months:
                filename = f"{t}_tripdata_2023-{m}.parquet"
                url = self.base_url + filename
                print(f"Baixando {filename} de {url}...")

                try:
                    r = requests.get(url, timeout=60)
                    r.raise_for_status()

                    local_tmp_path = f"/tmp/{filename}"
                    with open(local_tmp_path, "wb") as f:
                        f.write(r.content)

                    df = self.spark.read.parquet(local_tmp_path)
                    s3_output_path = f"{self.s3_base_path}/{t}/year=2023/month={m}"
                    print(f"Salvando no S3: {s3_output_path}")
                    df.write.mode("overwrite").parquet(s3_output_path)

                    os.remove(local_tmp_path)
                    print(" → OK")

                except requests.exceptions.HTTPError as e:
                    print(f"Arquivo indisponível ({e}), pulando {filename}.")
                except Exception as e:
                    print(f" Erro ao processar {filename}: {e}")


In [ ]:

gdrive_path = "/content/drive/MyDrive/ifood/teste_2/"
spark = SparkSession.builder.appName("iFoodCase").getOrCreate()

downloader = TripDataDownloader(
    spark=spark,
    base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/",
    s3_base_path=gdrive_path,
    years=[2023],
    types=['yellow', 'green', 'fhv', 'fhvhv'],
    months=['01', '02', '03', '04', '05']
)

downloader.download_and_save()
